In [2]:
import cv2

In [3]:
def generate_dataset(img, id, img_id):
    cv2.imwrite('data/users.'+str(id)+"."+str(img_id)+".jpg", img)

In [4]:
# Drawing a boundary for a feature
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    # Convert the image to grayscale to detect features efficiently
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Use the classifier to detect features in the grayscale image
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    # Initialize a list to hold the coordinates of detected features
    coords = []
    # Iterate over each detected feature
    for (x, y, w, h) in features:
        # Draw a rectangle around the detected feature on the original image
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        id, _ = clf.predict(gray_img[y:y+h, x:x+w])
        if id == 1:
            # Add text label to the detected feature
            cv2.putText(img, "SRIKAR", (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        # Store the coordinates of the detected feature
        coords = [x, y, w, h]
    # Return the coordinates of the detected feature
    return coords

def recognize(img, clf, faceCascade):
    color = {'blue':(255,0,0), 'red':(0,0,255), 'green':(0,255,0), 'white': (255,255,255)}

    coords = draw_boundary(img, faceCascade, 1.1, 10,color['white'],'Face',clf)

    return img
# Function to detect facial features in an image
def detect(img, faceCascade, eyesCascade, noseCascade, mouthCascade, img_id):
    # Dictionary to store color values for drawing boundaries
    color = {'blue': (255, 0, 0), 'red': (0, 0, 255), 'green': (0, 255, 0), 'white': (255, 255, 255)}
    # Call the draw_boundary function to draw boundaries around detected faces
    coords = draw_boundary(img, faceCascade, 1.1, 10, color['blue'], 'Face')

    # Check if a face is detected
    if len(coords) == 4:
        roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        
        user_id = 1
        generate_dataset(roi_img, user_id, img_id)
        # face detection
        # # Extract the region of interest (ROI) containing the detected face
        # roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        # # Draw boundaries around eyes, nose, and mouth within the ROI
        # coords = draw_boundary(roi_img, eyesCascade, 1.1, 10, color['white'], 'Eyes')
        # coords = draw_boundary(roi_img, noseCascade, 1.1, 5, color['green'], 'Nose')
        # coords = draw_boundary(roi_img, mouthCascade, 1.1, 20, color['white'], 'Mouth')
    # Return the modified image
    return img

# Load the cascade classifiers for face, eyes, nose, and mouth detection
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
eyesCascade = cv2.CascadeClassifier('haarcascade_eye.xml')
noseCascade = cv2.CascadeClassifier('Nariz.xml')
mouthCascade = cv2.CascadeClassifier('Mouth.xml')

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read('classifier.yml')

In [5]:
# Creating a video capture object to capture video from a webcam
video_capture = cv2.VideoCapture(0)  # If using the laptop's built-in webcam, use 0; otherwise, specify the appropriate webcam index (e.g., 1 or -1)

img_id = 0
# Continuously capture video frames and detect faces
while True:
    # Read a frame from the video capture object
    _, img = video_capture.read()
    
    # Call the detect function to detect faces in the captured frame
    #img = detect(img, faceCascade, eyesCascade, noseCascade, mouthCascade, img_id)
    
    img = recognize(img, clf, faceCascade)
    # Display the frame with face boundaries drawn
    cv2.imshow('Face detection', img)
    
    img_id += 1
    # Check if the user pressed 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object to free up resources
video_capture.release()

# Close all OpenCV windows
cv2.destroyAllWindows()
